In [1]:
import findspark
findspark.init()
findspark.find()

'/usr/local/opt/apache-spark/libexec'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
spark = (
    SparkSession
    .builder
    .appName("sparktest")
    .master("local[4]")
    .getOrCreate()
)
#A SparkContext represents the connection to a Spark cluster, and can be used to create RDD and broadcast variables on that cluster. It is the first step to using RDDs and connecting to a cluster

sc = spark.sparkContext
spark

23/07/13 11:29:21 WARN Utils: Your hostname, Hamsas-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.152 instead (on interface en0)
23/07/13 11:29:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/13 11:29:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
#can read csv using sc.textFile("C:/...")

In [12]:
#Apply map operation to split each record by comma, and get array of items
#lambda takes argument and an expression to evaluate

taxiZoneswithColsRdd = (
    taxiZonesRdd
        .map(lambda zone:zone.split(','))
)